<a href="https://colab.research.google.com/github/evansmakori/Bankruptcy-in-Poland/blob/main/05_5_bankruptcy_in_poland_Assignment_Bankruptcy_in_Taiwan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

5.5. Bankruptcy in Taiwan 🇹🇼

In [ ]:
import wqet_grader

wqet_grader.init("Project 5 Assessment")

In [ ]:
# Import libraries here
import gzip
import json
import pickle

import ipywidgets as widgets
import pandas as pd
import wqet_grader
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler
from IPython.display import VimeoVideo
from ipywidgets import interact
from sklearn.metrics import (
    ConfusionMatrixDisplay,
    classification_report,
    confusion_matrix,
)
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.pipeline import make_pipeline
from teaching_tools.widgets import ConfusionMatrixWidget

Prepare Data
Import
Task 5.5.1: Load the contents of the "data/taiwan-bankruptcy-data.json.gz" and assign it to the variable taiwan_data.

Note that taiwan_data should be a dictionary. You'll create a DataFrame in a later task.

# Load data file

In [ ]:
# Load data file
with gzip.open("data/taiwan-bankruptcy-data.json.gz","r") as f:
    taiwan_data=json.load(f)
print(type(taiwan_data))

Task 5.5.2: Extract the key names from taiwan_data and assign them to the variable taiwan_data_keys.

In [ ]:
taiwan_data_keys = taiwan_data.keys()
print(taiwan_data_keys)

Task 5.5.3: Calculate how many companies are in taiwan_data and assign the result to n_companies.

In [ ]:
n_companies = len(taiwan_data["observations"])
print(n_companies)

Task 5.5.4: Calculate the number of features associated with each company and assign the result to n_features.

In [ ]:
#taiwan_data["metadata"]
#taiwan_data["schema"].keys()

In [ ]:
n_features = len(taiwan_data["observations"][0])
print(n_features)

In [ ]:
df = pd.DataFrame().from_dict(taiwan_data["observations"])
print(df.shape)
df.head()

In [ ]:
for item in taiwan_data["observations"]:
    if len(item) !=97:
        print("ALERT")

Task 5.5.5: Create a wrangle function that takes as input the path of a compressed JSON file and returns the file's contents as a DataFrame. Be sure that the index of the DataFrame contains the ID of the companies. When your function is complete, use it to load the data into the DataFrame df.

In [ ]:
# Create wrangle function
def wrangle(filename):
    #open compressed file and load into dict
    with gzip.open(filename,"r") as f:
        taiwan_data= json.load(f)

    #tunring dict into DataFrame
    df=pd.DataFrame().from_dict(taiwan_data["observations"]).set_index("id")

    return df

In [ ]:
df = wrangle("data/taiwan-bankruptcy-data.json.gz")
print("df shape:", df.shape)
df.head()

Explore
Task 5.5.6: Is there any missing data in the dataset? Create a Series where the index contains the name of the columns in df and the values are the number of NaNs in each column. Assign the result to nans_by_col. Neither the Series itself nor its index require a name. WQU WorldQuant University Applied Data Science Lab QQQQ

In [ ]:
# Inspect DataFrame
#df.info()
#df.isnull().sum() / len(df)
len(df.isnull().sum())
df.info()

In [ ]:
nans_by_col = df.isnull().sum()

In [ ]:
nans_by_col = df.isnull().sum() / len(df)
pd.Series(nans_by_col)
print("nans_by_col shape:", nans_by_col.shape)

Task 5.5.7: Is the data imbalanced? Create a bar chart that shows the normalized value counts for the column df["bankrupt"]. Be sure to label your x-axis "Bankrupt", your y-axis "Frequency", and use the title "Class Balance".

In [ ]:
# Plot class balance
df["bankrupt"].value_counts(normalize=True).plot(
    kind="bar",
    xlabel="Bankrupt",
    ylabel="Frequency",
    title="Class Balance"
)
# Don't delete the code below 👇
plt.savefig("images/5-5-7.png", dpi=150)


Split
Task 5.5.8: Create your feature matrix X and target vector y. Your target is "bankrupt".

In [ ]:
target = "bankrupt"
X = df.drop(columns=target)
y = df[target]

print("X shape:", X.shape)
print("y shape:", y.shape)

Task 5.5.9: Divide your dataset into training and test sets using a randomized split. Your test set should be 20% of your data. Be sure to set random_state to 42.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X,y, test_size=0.2, random_state=42
)
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

Resample
Task 5.5.10: Create a new feature matrix X_train_over and target vector y_train_over by performing random over-sampling on the training data. Be sure to set the random_state to 42.

In [ ]:
over_sampler = RandomOverSampler(random_state=42)
X_train_over, y_train_over = over_sampler.fit_resample(X_train,y_train)
print("X_train_over shape:", X_train_over.shape)
X_train_over.head()

Build Model
Iterate
Task 5.5.11: Create a classifier clf that can be trained on (X_train_over, y_train_over). You can use any of the predictors you've learned about in the Data Science Lab.

Task 5.5.12: Perform cross-validation with your classifier using the over-sampled training data, and assign your results to cv_scores. Be sure to set the cv argument to 5.

In [ ]:
cv_scores = cross_val_score(
    clf, X_train_over, y_train_over, cv=5, n_jobs=-1
)
print(cv_scores)

Ungraded Task: Create a dictionary params with the range of hyperparameters that you want to evaluate for your classifier. If you're not sure which hyperparameters to tune, check the scikit-learn documentation for your predictor for ideas.

In [ ]:
params = {
    "n_estimators": range(25, 100, 25),
    "max_depth": range(10, 50, 10)
}
params

Task 5.5.13: Create a GridSearchCV named model that includes your classifier and hyperparameter grid. Be sure to set cv to 5, n_jobs to -1, and verbose to 1.

In [ ]:
model = GridSearchCV(
    clf,
    param_grid=params,
    cv=5,
    n_jobs=-1,
    verbose=1
)
model

Ungraded Task: Fit your model to the over-sampled training data.

In [ ]:
model.fit(X_train_over, y_train_over)

Task 5.5.14: Extract the cross-validation results from your model, and load them into a DataFrame named cv_results. Looking at the results, which set of hyperparameters led to the best performance?

In [ ]:
cv_results =pd.DataFrame(model.cv_results_)
cv_results.head(5)

Task 5.5.15: Extract the best hyperparameters from your model and assign them to best_params.

In [ ]:
best_params = model.best_params_
print(best_params)

Evaluate
Ungraded Task: Test the quality of your model by calculating accuracy scores for the training and test data.

In [ ]:
acc_train = model.score(X_train, y_train)
acc_test = model.score(X_test, y_test)

print("Model Training Accuracy:", round(acc_train, 4))
print("Model Test Accuracy:", round(acc_test, 4))

Task 5.5.16: Plot a confusion matrix that shows how your model performed on your test set.

In [ ]:
# Plot confusion matrix
ConfusionMatrixDisplay.from_estimator(model, X_test, y_test);
# Don't delete the code below 👇
plt.savefig("images/5-5-16.png", dpi=150)


Task 5.5.17: Generate a classification report for your model's performance on the test data and assign it to class_report.

In [ ]:
class_report = classification_report(y_test, model.predict(X_test))
print(class_report)

Communicate

In [ ]:
importances = model.best_estimator_

Task 5.5.18: Create a horizontal bar chart with the 10 most important features for your model. Be sure to label the x-axis "Gini Importance", the y-axis "Feature", and use the title "Feature Importance".

In [ ]:
# Get feature names from training data
features = X_train_over.columns
# Extract importances from model
importances = model.best_estimator_.named_steps["randomforestclassifier"].feature_importances_
# Create a series with feature names and importances
feat_imp = pd.Series(importances, index=features).sort_values()
# Plot 10 most important features
feat_imp.tail(10).plot(kind="barh")
plt.xlabel("Gini Importance")
plt.ylabel("Feature")
plt.title("Feature Importance");
# Don't delete the code below 👇
plt.savefig("images/5-5-17.png", dpi=150)


Task 5.5.19: Save your best-performing model to a a file named "model-5-5.pkl".

In [ ]:
# Save model
# Save model
with open("model-5-5.pkl", "wb") as f:
    pickle.dump(model, f)

Task 5.5.20: Open the file my_predictor_assignment.py. Add your wrangle function, and then create a make_predictions function that takes two arguments: data_filepath and model_filepath. Use the cell below to test your module. When you're satisfied with the result, submit it to the grader.



In [ ]:
# Import your module
def make_predictions(data_filepath, model_filepath):
    # Wrangle JSON file
    X_test = wrangle(data_filepath)
    # Load model
    with open(model_filepath, "rb") as f:
        model = pickle.load(f)

    # Generate predictions
    y_test_pred = model.predict(X_test)
    # Put predictions into Series with name "bankrupt", and same index as X_test
    y_test_pred = pd.Series(y_test_pred, index=X_test.index, name="bankrupt")
    return y_test_pred

# Generate predictions
y_test_pred = make_predictions(
    data_filepath="data/taiwan-bankruptcy-data-test-features.json.gz",
    model_filepath="model-5-5.pkl",
)

print("predictions shape:", y_test_pred.shape)
y_test_pred.head()

my Predict.py file

In [ ]:
# Create your masterpiece :)
# Import libraries
import gzip
import json
import pickle

import pandas as pd

def wrangle(filename):
    #open compressed file and load into dict
    with gzip.open(filename, "r") as f:
        data = json.load(f)
    #turn dict into dataframe
    df = pd.DataFrame().from_dict(data["data"]).set_index("company_id")
    return df


def make_predictions(data_filepath, model_filepath):
    # Wrangle JSON file
    X_test = wrangle(data_filepath)
    # Load model
    with open(model_filepath, "rb") as f:
        model = pickle.load(f)

    # Generate predictions
    y_test_pred = model.predict(X_test)
    # Put predictions into Series with name "bankrupt", and same index as X_test
    y_test_pred = pd.Series(y_test_pred, index=X_test.index, name="bankrupt")
    return y_test_pred


My predictor lesson py

In [ ]:
# Import libraries
import gzip
import json
import pickle

import pandas as pd


# Add wrangle function from lesson 5.4
def wrangle(filename):
    #open compressed file and load into dict
    with gzip.open(filename, "r") as f:
        data = json.load(f)
    #turn dict into dataframe
    df = pd.DataFrame().from_dict(data["data"]).set_index("company_id")
    return df


# Add make_predictions function from lesson 5.3
def make_predictions(data_filepath, model_filepath):
    # Wrangle JSON file
    X_test = wrangle(data_filepath)
    # Load model
    with open(model_filepath, "rb") as f:
        model = pickle.load(f)

    # Generate predictions
    y_test_pred = model.predict(X_test)
    # Put predictions into Series with name "bankrupt", and same index as X_test
    y_test_pred = pd.Series(y_test_pred, index=X_test.index, name="bankrupt")
    return y_test_pred


# New Section